# IMPORTS

In [90]:
import pandas as pd
import random
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import os
import time

# DATA PREP

In [ ]:
# Download and unzip metadatiia if necessary, then jump into correct directory to keep all other paths valid
if not os.path.exists("./fma_metadata.zip"):
    os.system("wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip")
if not os.path.exists("./fma_metadata/tracks.csv") or not os.path.exists("./fma_metadata/features.csv"):
    os.system("unzip fma_metadata.zip")
%cd fma_metadata
# Optional if you dont have the feature.csv and tracks.csv in the folder, only tested on Ubuntu

In [91]:
# Load tracks
df_tracks = pd.read_csv("tracks.csv",index_col=0, header=[0, 1])

# Filter tracks so that they correspond to the "medium" dataset
df_tracks = df_tracks[df_tracks["set"]["subset"] == "medium"]

# Filter further based on genres (there are 16 in total and we only need 8 of them)
genres = ["Hip-Hop", "Pop", "Folk", "Rock", "Experimental", "International", "Electronic", "Instrumental"]
df_tracks = df_tracks[df_tracks["track"]["genre_top"].isin(genres)]

# Split the data
train_tracks, val_tracks, test_tracks = df_tracks[df_tracks["set"]["split"] == "training"], df_tracks[df_tracks["set"]["split"] == "validation"], df_tracks[df_tracks["set"]["split"] == "test"]

In [92]:
# Load features
df_features = pd.read_csv("features.csv", index_col=0, header=[0, 1, 2])

# Get feature data for each set
train_features = df_features.loc[train_tracks.index]
val_features = df_features.loc[val_tracks.index]
test_features = df_features.loc[test_tracks.index]

scaler = StandardScaler()

# Standardize
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)
test_features_scaled = scaler.transform(test_features)

# We need the dataframes for the indexing we use later
train_features_scaled = pd.DataFrame(train_features_scaled, index=train_features.index, columns=train_features.columns)
val_features_scaled = pd.DataFrame(val_features_scaled, index=val_features.index, columns=val_features.columns)
test_features_scaled = pd.DataFrame(test_features_scaled, index=test_features.index, columns=test_features.columns)

In [114]:
FEATURES_PER_TRACK = train_features.shape[1]
L, N, K = 8, 4, 15
M = "cosine" 

# CALCULATE HASH TABLES

In [102]:
# Function for generating the R matrix
def create_R(l, d):
    R = np.zeros((l, d))
    distribution = [1, -1, 0, 0, 0, 0] # 1 -> 1/6, -1 -> 1/6, 0 -> 2/3
    for i in range(l):
        for j in range(d):
            R[i][j] = np.sqrt(3) * distribution[random.randint(0, 5)]
    return R

# Function for hash calculation
def calculate_hash(track_feature, R):
    # X needs to be FEATURES_PRE_TRACK x 1
    track_feature = np.array(track_feature).reshape(-1, 1)
    return np.sign(np.dot(R, track_feature))

In [110]:
hashtables = [{} for _ in range(N)]
R_matrices = [create_R(L, FEATURES_PER_TRACK) for _ in range(N)]
# Go through all the hashtables
for i in range(N):
    # We need to store the projection matrices we used in the correct order for later use
    R = R_matrices[i]
    # For each track, we will calculate it's hash value, for i-th hashtable
    for track_id, track in train_tracks.iterrows(): 
        track_feature = train_features_scaled.loc[track_id] 
        f_hash_value = calculate_hash(track_feature, R)
        # We flatten so we can use the hash value in this form -> (1,1,-1,...,-1) as the hashkey
        f_hash_key = tuple(f_hash_value.flatten()) 
        if f_hash_key not in hashtables[i]:
            hashtables[i][f_hash_key] = []
        hashtables[i][f_hash_key].append(track_id)


# FIND SIMILAR TRACKS FOR THE VAL DATASET

In [104]:
def find_similar_for_val(val_track_id, val_track_feature, R_matrices):
    # We use a set because the same candidate track may appear in multiple hashtables
    similar_tracks = set()
    # We go through all the hashtables
    for i in range(len(hashtables)):
        # We need to use the same R we used when creating each hashtable
        hashtable = hashtables[i]
        R = R_matrices[i]
        vf_hash_value = calculate_hash(val_track_feature, R)
        vf_hash_key = tuple(vf_hash_value.flatten())
        # If a the hashtable has a key same as vf_hash_key, then all the values there are candidate similar tracks to the val_track
        if vf_hash_key in hashtable:
            similar_tracks.update(hashtable[vf_hash_key])
    return (val_track_id, list(similar_tracks))

In [111]:
# For each one of the validation tracks, find the train tracks that are similar to it in atleast one hashtable
val_similarity_dic = {}
for i in range(len(val_tracks)):
    track_id = val_features_scaled.index[i]
    feature = val_features_scaled.loc[track_id]
    sim_tuple = find_similar_for_val(track_id, feature, R_matrices)
    val_similarity_dic[sim_tuple[0]] = sim_tuple[1]

print(len(val_similarity_dic))

1495


# CALCULATE ACTUAL SIMILARITY FOR TRACKS IN THE SAME BUCKET

In [106]:
# Possible metrics: "cosine", "euclidean" (default cosine, pivots to euclidean if invalid input value)
def calculate_k_nearest(val_track_feature, candidate_track_ids, k, metric="cosine"):
    global train_features_scaled
    
    similarity = lambda a, b : -cosine_similarity([a], [b])[0][0]

    if metric != "cosine":
        similarity = lambda a, b : np.linalg.norm(a - b)
    
    candidate_track_ids_set = set(candidate_track_ids)

    distances = []

    for track_id in candidate_track_ids_set:
        distances.append((track_id, similarity(val_track_feature,train_features_scaled.loc[track_id])))

    distances = sorted(distances, key=lambda x : x[1])
    
    return [entry[0] for entry in distances[:k]]


# COMPUTE THE VALIDATION PREDICTIONS

In [112]:
# Calculate the k nearest neighboors for all validation data, according to both cosine and euclidean distance
k_nearest_cos = {}
for track_id, track in val_tracks.iterrows():
    track_feature = val_features_scaled.loc[track_id]
    candidates = val_similarity_dic[track_id]
    nearest = calculate_k_nearest(track_feature, candidates, K, "cosine")
    nearest_ids = [near for near in nearest]
    k_nearest_cos[track_id] = nearest_ids

k_nearest_euc = {}
for track_id, track in val_tracks.iterrows():
    track_feature = val_features_scaled.loc[track_id]
    candidates = val_similarity_dic[track_id]
    nearest = calculate_k_nearest(track_feature, candidates, K, "euclidean")
    nearest_ids = [near for near in nearest]
    k_nearest_euc[track_id] = nearest_ids

In [113]:
validation_correct = 0
for val_id, similar_ids in k_nearest_cos.items():
    # We will use this dictionary to hold the number of appearances of each genre in the specific simila_ids list
    genre_count = {"Hip-Hop":0, "Pop":0, "Folk":0, "Rock":0, "Experimental":0, "International":0, "Electronic":0, "Instrumental":0}
    # Similar_ids are the k nearest train track ids to the validation track with id = val_id
    for id in similar_ids:
        track = train_tracks.loc[id]
        track_genre = track[("track", "genre_top")]
        genre_count[track_genre] += 1
    # Find the genre that appeard the most amongst the tracks in similar_ids and compare it to the true genre of the val track
    most_common_genre = max(genre_count, key=genre_count.get)
    if (most_common_genre == val_tracks.loc[val_id][("track", "genre_top")]):
        validation_correct += 1
print(validation_correct/len(test_tracks))

validation_correct = 0
for val_id, similar_ids in k_nearest_euc.items():
    # We will use this dictionary to hold the number of appearances of each genre in the specific simila_ids list
    genre_count = {"Hip-Hop":0, "Pop":0, "Folk":0, "Rock":0, "Experimental":0, "International":0, "Electronic":0, "Instrumental":0}
    # Similar_ids are the k nearest train track ids to the validation track with id = val_id
    for id in similar_ids:
        track = train_tracks.loc[id]
        track_genre = track[("track", "genre_top")]
        genre_count[track_genre] += 1
    # Find the genre that appeard the most amongst the tracks in similar_ids and compare it to the true genre of the val track
    most_common_genre = max(genre_count, key=genre_count.get)
    if (most_common_genre == val_tracks.loc[val_id][("track", "genre_top")]):
        validation_correct += 1
print(validation_correct/len(test_tracks))




0.6104234527687297
0.5928338762214984


# TEST ACCURACY CALCULATION

In [ ]:
# Combine the training and validation data before scaling
train_val_features = pd.concat([train_features, val_features])
scaler = StandardScaler()
train_val_features_scaled = scaler.fit_transform(train_val_features)
test_features_scaled = scaler.transform(test_features)

# Convert back to DataFrame for indexing
train_val_features_scaled = pd.DataFrame(train_val_features_scaled, index=train_val_features.index, columns=train_val_features.columns)
test_features_scaled = pd.DataFrame(test_features_scaled, index=test_features.index, columns=test_features.columns)

# Calculate the hashtables based on the train_val features
hashtables = [{} for _ in range(N)]
R_matrices = [create_R(L, FEATURES_PER_TRACK) for _ in range(N)]
# Go through all the hashtables
for i in range(N):
    # We need to store the projection matrices we used in the correct order for later use
    R = R_matrices[i]
    # For each track, we will calculate it's hash value, for i-th hashtable
    for track_id, track in train_tracks.iterrows(): 
        track_feature = train_val_features_scaled.loc[track_id] 
        f_hash_value = calculate_hash(track_feature, R)
        # We flatten so we can use the hash value in this form -> (1,1,-1,...,-1) as the hashkey
        f_hash_key = tuple(f_hash_value.flatten()) 
        if f_hash_key not in hashtables[i]:
            hashtables[i][f_hash_key] = []
        hashtables[i][f_hash_key].append(track_id)

def find_similar_for_val(val_track_id, val_track_feature, R_matrices):
    # We use a set because the same candidate track may appear in multiple hashtables
    similar_tracks = set()
    # We go through all the hashtables
    for i in range(len(hashtables)):
        # We need to use the same R we used when creating each hashtable
        hashtable = hashtables[i]
        R = R_matrices[i]
        vf_hash_value = calculate_hash(val_track_feature, R)
        vf_hash_key = tuple(vf_hash_value.flatten())
        # If a the hashtable has a key same as vf_hash_key, then all the values there are candidate similar tracks to the val_track
        if vf_hash_key in hashtable:
            similar_tracks.update(hashtable[vf_hash_key])
    return (val_track_id, list(similar_tracks))

test_similarity_dic = {}
for i in range(len(test_tracks)):
    track_id = test_features_scaled.index[i]
    feature = test_features_scaled.loc[track_id]
    sim_tuple = find_similar_for_val(track_id, feature, R_matrices)
    test_similarity_dic[sim_tuple[0]] = sim_tuple[1]

def calculate_k_nearest(val_track_feature, candidate_track_ids, k, metric="cosine"):
    similarity = lambda a, b : -cosine_similarity([a], [b])[0][0]

    if metric != "cosine":
        similarity = lambda a, b : np.linalg.norm(a - b)
    
    candidate_track_ids_set = set(candidate_track_ids)

    distances = []

    for track_id in candidate_track_ids_set:
        distances.append((track_id, similarity(val_track_feature,train_val_features_scaled.loc[track_id])))

    distances = sorted(distances, key=lambda x : x[1])
    
    return [entry[0] for entry in distances[:k]]

k_nearest = {}
for track_id, track in test_tracks.iterrows():
    track_feature = test_features_scaled.loc[track_id]
    candidates = test_similarity_dic[track_id]
    nearest = calculate_k_nearest(track_feature, candidates, K, M)
    nearest_ids = [near for near in nearest]
    k_nearest[track_id] = nearest_ids

test_correct = 0
for test_id, similar_ids in k_nearest.items():
    # We will use this dictionary to hold the number of appearances of each genre in the specific simila_ids list
    genre_count = {"Hip-Hop":0, "Pop":0, "Folk":0, "Rock":0, "Experimental":0, "International":0, "Electronic":0, "Instrumental":0}
    # Similar_ids are the k nearest train track ids to the test track with id = test_id
    for id in similar_ids:
        track = train_tracks.loc[id]
        track_genre = track[("track", "genre_top")]
        genre_count[track_genre] += 1
    # Find the genre that appeard the most amongst the tracks in similar_ids and compare it to the true genre of the test track
    most_common_genre = max(genre_count, key=genre_count.get)
    if (most_common_genre == test_tracks.loc[test_id][("track", "genre_top")]):
        test_correct += 1
print(test_correct/len(test_tracks))

# 66.18%

0.6618892508143323
